In [1]:
from Bio.Blast import NCBIWWW
from Bio.Blast import NCBIXML
import grakn
import pandas as pd
import numpy as np
import random


In [18]:
# Generate a list of subsequences to draw from BLAST
Ntrials = 500

sequence = "MAEVPELASEMMAYYSGNEDDLFFEADGPKQMKCSFQDLDLCPLDGGIQLR \
            ISDHHYSKGFRQAASVVVAMDKLRKMLVPCPQTFQENDLSTFFPFIFEEEP \
            IFFDTWDNEAYVHDAPVRSLNCTLRDSQQKSLVMSGPYELKALHLQGQDME \
            QQVVFSMSFVQGEESNDKIPVALGLKEKNLYLSCVLKDDKPTLQLESVDPK \
            NYPKKKMEKRFVFNKIEINNKLEFESAQFPNWYISTSQAENMPVFLGGTKG \
            GQDITDFTMQFVSS"

subseqs = []
for i in range(173, Ntrials):
    print("Blast {i}".format(i=i))
    subseqLen = random.randint(20, 50)
    seq_init = random.randint(0, len(sequence) - subseqLen)
    sub_seq = sequence[seq_init:seq_init + subseqLen]
    subseqs.append(sub_seq)
    if i%2 == 0:
        result_handle = NCBIWWW.qblast("blastp", "nr", sub_seq, hitlist_size=200)
    else:
        result_handle = NCBIWWW.qblast("blastp", "nr", sub_seq, hitlist_size=200, service="psi", threshold=1000)
    with open("blast{i}.xml".format(i=i), "w") as out_handle:
        out_handle.write(result_handle.read())

    result_handle.close()

print(subseqs)

In [19]:
# Build the table

col_prot = ["name", "id", "species", "query_seq", "query_number", "identicality", "positivity", "gaps", "midline"]
protDB = pd.DataFrame(columns=col_prot)

# Go over all BLAST result files and store them in the table
for i in range(132, 157):
    result_handle = open("Blast{i}.xml".format(i=i))
    blast_record = NCBIXML.read(result_handle)

    for alignment in blast_record.alignments:
        # initiate a new row in table
        protRow = len(protDB)
        protDB.loc[protRow] = [0 for _ in range(len(protDB.columns))]
        
        # Store all info in the table
        protDB.name[protRow] = alignment.hit_def.split(" >")[0].split(";")[0]
        protDB.id[protRow] = alignment.hit_id.split("|", 4)[3]
        if (len(alignment.hit_def.split("[")) > 1):
            protDB.species[protRow] = alignment.hit_def.split("[")[1].split("]")[0]
        for hsp in alignment.hsps:
            protDB.positivity[protRow] = round(hsp.positives / alignment.length, 3)
            protDB.identicality[protRow] = round(hsp.identities / alignment.length, 3)
            protDB.gaps[protRow] = round(hsp.gaps / alignment.length, 5)
            protDB.midline[protRow] = hsp.match
        protDB.query_number[protRow] = i
    print("Done with {i}".format(i=i))

protDB.to_excel("Full_BLUST_results.xlsx")
protDB.head()

,name,id,species,query_seq,query_number,identicality,positivity,gaps,midline
0,"Chain A, Interleukin-1 Beta (Il-1 Beta) (Mutan...",21BI,0,0,0,0.209,0.209,0,PYELKALHLQGQDMEQQVVFSMSFVQGEESND
1,"Chain A, Interleukin-1 Beta (Il-1 Beta) (Mutan...",41BI,0,0,0,0.209,0.209,0,PYELKALHLQGQDMEQQVVFSMSFVQGEESND
2,"Chain A, Interleukin-1 Beta (Il-1 Beta) (Mutan...",31BI,0,0,0,0.209,0.209,0,PYELKALHLQGQDMEQQVVFSMSFVQGEESND
3,"Chain A, INTERLEUKIN 1 BETA MUTANT F101Y",1TWE,0,0,0,0.209,0.209,0,PYELKALHLQGQDMEQQVVFSMSFVQGEESND
4,"Chain A, Interleukin-1 Beta Mutant F146y",1TWM,0,0,0,0.209,0.209,0,PYELKALHLQGQDMEQQVVFSMSFVQGEESND


In [13]:
filtered_protDB = protDB[~protDB['name'].str.contains("interleukin-1 beta")]
filtered_protDB = filtered_protDB[~filtered_protDB['name'].str.contains("Interleukin-1 beta")]
filtered_protDB = filtered_protDB[~filtered_protDB['name'].str.contains("interleukin 1 beta")]
filtered_protDB = filtered_protDB[~filtered_protDB['name'].str.contains("interleukin-1beta")]
filtered_protDB = filtered_protDB[~filtered_protDB['name'].str.contains("interleukin-1")]
filtered_protDB = filtered_protDB[~filtered_protDB['name'].str.contains("interleukin-1-B")]
filtered_protDB = filtered_protDB[~filtered_protDB['name'].str.contains("interleukin 1b")]
filtered_protDB = filtered_protDB[~filtered_protDB['name'].str.contains("interleukin 1, beta")]
filtered_protDB = filtered_protDB[~filtered_protDB['name'].str.contains("IL-1 beta")]
filtered_protDB = filtered_protDB[~filtered_protDB['name'].str.contains("IL1B")]
filtered_protDB = filtered_protDB[~filtered_protDB['name'].str.contains("INTERLEUKIN-1 BETA")]
filtered_protDB = filtered_protDB[~filtered_protDB['name'].str.contains("INTERLEUKIN 1 BETA")]
filtered_protDB = filtered_protDB[~filtered_protDB['name'].str.contains("Interleukin-1 Beta")]
filtered_protDB = filtered_protDB[~filtered_protDB['name'].str.contains("Il-1 Beta")]
filtered_protDB = filtered_protDB[~filtered_protDB['name'].str.contains("Il-1B")]
filtered_protDB = filtered_protDB[~filtered_protDB['name'].str.contains("Il-1b")]
filtered_protDB = filtered_protDB[~filtered_protDB['name'].str.contains("IL-1b")]
filtered_protDB = filtered_protDB[~filtered_protDB['name'].str.contains("IL-1beta")]
filtered_protDB = filtered_protDB[~filtered_protDB['name'].str.contains("interleukin-1b")]

filtered_protDB = filtered_protDB.drop_duplicates(subset='name')
filtered_protDB.to_excel("Filtered_BLUST_results.xlsx")